# Lab

Update the todo list application to allow editing a todo item (you will probably need another template)

In [5]:
%%file data/flask-examples/todo-soln.py
import os
from flask import Flask, render_template, request, url_for, redirect

from .state import StateManager

app = Flask(__name__)

STATE_MANAGER = StateManager(
    os.path.abspath(
        os.path.join(
            os.path.dirname(__file__),
            'state.pkl'
        )
    )
)

@app.route('/')
def home():
    state = STATE_MANAGER.get()
    items = state.get('items', [])
    return render_template('todo2.html', items=items)


@app.route('/item', methods=['POST'])
def add_todo():
    state = STATE_MANAGER.get()
    items = state.setdefault('items', [])
    items.append(request.form['item'])
    STATE_MANAGER.save()
    return redirect(url_for('home'))


@app.route('/item/<int:index>', methods=['POST'])
def update_todo(index):
    state = STATE_MANAGER.get()
    items = state.get('items', [])
    if index < len(items):
        if 'delete' in request.form:
            del items[index]
        else:
            items[index] = request.form['item']
        STATE_MANAGER.save()
    return redirect(url_for('home'))


Overwriting data/flask-examples/todo-soln.py


In [6]:
%%file data/flask-examples/templates/todo2.html
<!doctype html>
<title>Simple Todo</title>

<h1>To-Do List</h1>

<ul>
{% for item in items %}
    <li>
        <form method="POST" action="{{url_for('update_todo', index=loop.index0)}}">
            <input name="item" value="{{item}}"/><br/>
            <input type="submit" value="Save"/>
            <input type="submit" name="delete" value="Delete"/>
        </form>
    </li>
{% endfor%}
</ul>

<form method="POST" action={{url_for('add_todo')}}>
    <label for="item">Add todo item</label><br/>
    <input name="item"/>
</form>


Overwriting data/flask-examples/templates/todo2.html


In [7]:
import os, sys, time, threading, subprocess, contextlib

def output_thread(proc):
    for line in proc.stdout:
        print(line.decode('utf-8'), end='')
    print('Exiting output thread')

def run_flask_app(app_name):
    proc = subprocess.Popen(
        # [sys.executable, 'flask', 'run'],
        ['flask', 'run', '--no-reload'],
        env={
            **os.environ, 
            'FLASK_APP': app_name,
            'FLASK_ENV': 'development',
        },
        stderr=subprocess.STDOUT,
        stdout=subprocess.PIPE
    )
    # Wait for the port to bind
    for line in proc.stdout:
        line = line.decode('utf-8')
        print(line, end='')
        if ' * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)' in line:
            break
    else:
        print('== Error starting server ==')
        return None
    thd = threading.Thread(target=output_thread, args=(proc,))
    thd.setDaemon(True)
    thd.start()
    return proc


@contextlib.contextmanager
def running_app(app_name):
    proc = run_flask_app(app_name)
    try:
        yield proc
    finally:
        proc.kill()        

In [8]:
sp = run_flask_app('data.flask-examples.todo-soln')

 * Serving Flask app "data.flask-examples.todo-soln"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 13:50:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:50:54] "POST /item HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:50:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:50:58] "POST /item HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:50:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:51:00] "POST /item HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:51:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:51:05] "POST /item/1 HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:51:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:51:11] "POST /item/2 HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:51:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:51:13] "POST /item/0 HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:51:13] "GET / HTTP/1.1" 200 -


In [5]:
sp.kill()

Exiting output thread
